In [1]:
from collections import Counter
import re, string, random, glob, operator, heapq
from collections import defaultdict
from math import log10
from functools import reduce
from functools import lru_cache

In [2]:
COCT = open('COCT.small.txt', encoding="utf8").read().splitlines()

In [3]:
words = []
for i in range(len(COCT)):
    word = [x.split('|') for x in COCT[i].split(' ')]
    
    for j in range(len(word)):
        words.append(word[j][0])

In [4]:
word_counts = Counter(words)
word_counts

Counter({'但是': 2909,
         '男性': 358,
         '在': 63466,
         '他們': 11280,
         '優越': 34,
         '的': 330281,
         '、': 57659,
         '專權': 22,
         '世界': 2860,
         '裡': 14211,
         '，': 608690,
         '竟': 1314,
         '也': 37575,
         '充滿': 914,
         '了': 96009,
         '壓力': 729,
         '矛盾': 282,
         '和': 27922,
         '迷惘': 34,
         '角色': 778,
         '是': 108361,
         '不': 60224,
         '容易': 1614,
         '扮演': 301,
         '。': 260141,
         '女權': 30,
         '運動': 1096,
         '興起': 145,
         '以後': 2090,
         '腳步': 266,
         '亂': 868,
         '也許': 1182,
         '會': 17518,
         '交': 268,
         '一': 93291,
         '個': 53545,
         '成績': 438,
         '很': 15276,
         '好': 11491,
         '女朋友': 87,
         '希望': 2384,
         '課業': 43,
         '上': 30474,
         '能': 12680,
         '得到': 1690,
         '她': 25505,
         '幫助': 1084,
         '博士': 521,
         '還':

In [20]:
tokens = {}
N = 0
for k,v in word_counts.items():
    name, count = k, int(v)
    N += count
    tokens[name] = count

    for j in range(len(name)):
        t = name[: j + 1]
        if t not in tokens:
            tokens[t] = 0

In [22]:
word_counts = tokens

In [23]:
N = 1024908267229 ## Number of tokens
V = len(word_counts)

In [24]:
def build_tree(sentence):
    tree = []
    for idx, word in enumerate(sentence):
        l = len(sentence)

        words = {}

        for i in range(1, l - idx + 1):
            token = sentence[idx : idx + i]

            value = word_counts.get(token)
            if value != None:
                if value != 0:
                    words[idx + i - 1] = word_counts[token]
                else:
                    continue
            else:
                #words[idx + i - 1] = word_counts[token]
                break

        #if len(words) > 2:
        #    words.popitem()
                
        tree.append(words)
    return tree

In [25]:
build_tree("今天可能下雨")

[{0: 813, 1: 1580},
 {1: 10831},
 {2: 7060, 3: 4280},
 {3: 12680},
 {4: 8289, 5: 106},
 {5: 455}]

In [26]:
def segment(text):
    "Return a list of words that is the best segmentation of text."
    if not text: return []
    tree = build_tree(text)
    
    @lru_cache(None)
    def dp(index=0):
        possibles = []
        if index >= len(text):
            return [log10(1), []]
        for i in tree[index]:
            possibles.append([text[index : i + 1], text[i + 1 :]])
            
        #print(possibles)

        prob_possibles = []
        for i in possibles:
            other = dp(index=len(text) - len(i[1]))

            try:
                prob = log10((word_counts[i[0]]+1) / (N+V)) + other[0]
            except:
                print("debug:", (word_counts[i[0]]+1) / (N+V))
                print("debug:", other[0])
            possible = [i[0]] + other[1]
            prob_possibles.append([prob, possible])

        m = max(prob_possibles, key=lambda x: x[0])

        return m
    
    return dp(0)
print ('done')

done


In [27]:
sentences = [
    "今天可能下雨",
    "中華民國於一九九五年三月一日開始實施全民健保",
    "孫中山先生推翻滿清建立中華民國",
    "教授應該要為辛苦的助教加薪",
    "火鍋是四川的特色",
    "波士頓茶葉事件促使美國革命",
    "羅馬帝國皇帝遭到殺害",
]

In [28]:
for i in sentences:
    print(segment(i))

[-27.172194496178108, ['今天', '可能', '下雨']]
[-77.43747890747946, ['中華民國', '於', '一九九五年', '三月', '一日', '開始', '實施', '全民健保']]
[-56.849396415706266, ['孫中山', '先生', '推翻', '滿清', '建立', '中華民國']]
[-70.89204642623014, ['教授', '應該', '要', '為', '辛苦', '的', '助教', '加薪']]
[-43.58146345668928, ['火鍋', '是', '四川', '的', '特色']]
[-57.37573847167542, ['波士頓', '茶葉', '事件', '促使', '美國', '革命']]
[-46.72274566130841, ['羅馬', '帝國', '皇帝', '遭到', '殺害']]


In [29]:
segment('中產階級在一九九五年發動政變')

[-47.52280365907019, ['中產階級', '在', '一九九五年', '發動', '政變']]

In [30]:
segment('七步成詩是一句成語')

[-68.63402981426137, ['七', '步', '成', '詩', '是', '一', '句', '成語']]